# Correction des typos

Le but de ce projet est de corriger les fautes de frappes dans un texte sans recours à un dictionnaire, en utilisant un modèle de Markov caché (HMM).

# I - Chargement des données

Données issues du *Manifeste de l'Unabomber* et artificiellement bruitées en mofifiant aléatoirement 10% ou 20% des lettres du corpus (erreurs de substitutions seulement).

In [1]:
import numpy as np

from HMM import *
from toolbox import *

In [2]:
# Import and separate datasets
ERROR_RATE = 10  # 10% or 20%
train_set, test_set = load_db(error_rate=ERROR_RATE)
X_train = [[token[0] for token in word] for word in train_set]
y_train = [[token[1] for token in word] for word in train_set]
X_test = [[token[0] for token in word] for word in test_set]
y_test = [[token[1] for token in word] for word in test_set]

# Get states and observations sets
states, observations = get_observations_states(X_train, y_train)
print("{} states :\n{}".format(len(states), states))
print("{} observations :\n{}".format(len(observations), observations))

# Example from dataset
print("\nSample example (observation, état) :\n{}".format(train_set[3]))

26 states :
['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
26 observations :
['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

Sample example (observation, état) :
[('a', 'a'), ('c', 'c'), ('v', 'c'), ('o', 'o'), ('u', 'u'), ('n', 'n'), ('t', 't')]


# II - HMM d'ordre 1

Essai de correction des typos par un HMM d'ordre 1 utilisant l'algorithme de Viterbi.
On prend en compte un seul instant précédent pour le calcul de la transition. On considère donc les probabilités :
- $P(X_t| Y_t)$ : probabilité d'observation de X à l'état Y à l'instant t
- $P(Y_t| Y_{t-1})$ : probabilité de l'état Y à l'instant t sachant  l'état précédent

Afin d'éviter les probabilités nulles, on initialise toutes les matrices (émission, transition et état initial) par une probabilité uniforme (respectivement $1/n_{observations}$, $1/n_{states}$, $1/n_{states}$), puis on affine ces probabilités grâce aux comptes selon le maximum de vraisemblance.

In [3]:
# Initialize and train HMM
hmm1 = HMM(states, observations)
hmm1.fit(X_train, y_train)

1st order HMM created with: 
 * 26 states
 * 26 observations
Training initial states probabilities... Done.
Training transitions probabilities given states... Done.
Training observations probabilities given states... Done.


In [4]:
# Try HMM prediction on one sample
SAMPLE = 11
observation_sequence = X_test[SAMPLE]
states_sequence = y_test[SAMPLE]

predicted_states_sequence = hmm1.predict([observation_sequence])

print("Observation sequence      : {}".format("".join(observation_sequence)))
print("Real states sequence      : {}".format("".join(states_sequence)))
print("Predicted states sequence : {}".format("".join(predicted_states_sequence[0])))

Observation sequence      : inferikrigy
Real states sequence      : inferiority
Predicted states sequence : inderiorigy


In [5]:
y_test_pred = hmm1.predict(X_test)
display_correction_stats(X_test, y_test, y_test_pred, name="HMM1")

HMM1 score on test set
 * accuracy on full words : 75.15%
 * accuracy on letters    : 93.20%
   > typos corrected      : 310 (4.23%)
   > typos not corrected  : 435 (5.94%)
   > typos added          : 63 (0.86%)

Dummy score on test set
 * accuracy on full words : 62.89%
 * accuracy on letters    : 89.82%
   > typos corrected      : 0 (0.00%)
   > typos not corrected  : 745 (10.18%)
   > typos added          : 0 (0.00%)



# III -HMM d'ordre 2

## HMM d'ordre 2

On prend en compte les deux instants précédents pour le calcul de la transition. On considère donc maitenant les probabilités :
- $P(X_t| Y_t)$ : probabilité d'observation de X à l'état Y à l'instant t (identique à l'ordre 1)
- $P(Y_t| Y_{t-1}, Y_{t-2})$ : probabilité de l'état Y à l'instant t sachant les deux états précédents.

La dimension de la matrice de transition étant plus importante, certains trigrammes peuvent ne pas avoir été observées dans le corpus d'apprentissage. Afin d'éviter les probabilités nulles, on initialise donc toutes les matrices (émission, transition et état initial) par une probabilité uniforme (respectivement $1/n_{observations}$, $1/n_{states}$, $1/n_{states}$), puis on affine ces probabilités grâce aux comptes selon le maximum de vraisemblance.

In [6]:
# Initialize and train HMM
hmm2 = HMM2(states, observations)
hmm2.fit(X_train, y_train)

2nd order HMM created with: 
 * 26 states
 * 26 observations
Training initial states probabilities... Done.
Training transitions probabilities given states... Done.
Training observations probabilities given states... Done.


In [7]:
# Try HMM prediction on one sample
SAMPLE = 11
observation_sequence = X_test[SAMPLE]
states_sequence = y_test[SAMPLE]

predicted_states_sequence = hmm2.predict([observation_sequence])

print("Observation sequence      : {}".format("".join(observation_sequence)))
print("Real states sequence      : {}".format("".join(states_sequence)))
print("Predicted states sequence : {}".format("".join(predicted_states_sequence[0])))

Observation sequence      : inferikrigy
Real states sequence      : inferiority
Predicted states sequence : inferiority


In [8]:
y_test_pred = hmm2.predict(X_test)
display_correction_stats(X_test, y_test, y_test_pred, name="HMM2")

HMM2 score on test set
 * accuracy on full words : 83.41%
 * accuracy on letters    : 95.57%
   > typos corrected      : 513 (7.01%)
   > typos not corrected  : 232 (3.17%)
   > typos added          : 92 (1.26%)

Dummy score on test set
 * accuracy on full words : 62.89%
 * accuracy on letters    : 89.82%
   > typos corrected      : 0 (0.00%)
   > typos not corrected  : 745 (10.18%)
   > typos added          : 0 (0.00%)


## HMM d'ordre 2 avec smoothing

Comme de nombreux trigrammes peuvent ne pas avoir été rencontrés dans le corpus d'apprentissage, on peut introduire différentes techniques de lissage ou d'extrapolation des données. On teste ici la technique introduite par [cet article](http://www.aclweb.org/anthology/P99-1023).

In [9]:
# Initialize and train HMM
hmm2s = HMM2(states, observations)
hmm2s.fit(X_train, y_train, smoothing='weighted')

2nd order HMM created with: 
 * 26 states
 * 26 observations
Training initial states probabilities... Done.
Training transitions probabilities given states... Done.
Training observations probabilities given states... Done.


In [10]:
# Try HMM prediction on one sample
SAMPLE = 11
observation_sequence = X_test[SAMPLE]
states_sequence = y_test[SAMPLE]

predicted_states_sequence = hmm2s.predict([observation_sequence])

print("Observation sequence      : {}".format("".join(observation_sequence)))
print("Real states sequence      : {}".format("".join(states_sequence)))
print("Predicted states sequence : {}".format("".join(predicted_states_sequence[0])))

Observation sequence      : inferikrigy
Real states sequence      : inferiority
Predicted states sequence : inferiorigy


In [11]:
y_test_pred = hmm2s.predict(X_test)
display_correction_stats(X_test, y_test, y_test_pred, name="HMM2_smooth")

HMM2_smooth score on test set
 * accuracy on full words : 79.61%
 * accuracy on letters    : 94.66%
   > typos corrected      : 406 (5.55%)
   > typos not corrected  : 339 (4.63%)
   > typos added          : 52 (0.71%)

Dummy score on test set
 * accuracy on full words : 62.89%
 * accuracy on letters    : 89.82%
   > typos corrected      : 0 (0.00%)
   > typos not corrected  : 745 (10.18%)
   > typos added          : 0 (0.00%)


# IV - Insertion de caractères

Pour simuler l'insertion de caractères, on ajoute avec un certaine probabilité un caractère (dont la touche du clavier est proche de la précédente). Comme cette observation supplémentaire ne correspond pas un état (pas de véritable lettre associée à cette insertion), on modélise celui-ci par un nouvel état '\_'. On se retrouve face à un problème de 27 états (26 lettres + '\_') et 26 observations.

Exemple:
- **séquence d'états        :** ['a', 'c', 'c', 'o', '\_', 'u', 'n', 't']
- **séquence d'observations :** ['a', 'c', 'v', 'o', 'p', 'u', 'n', 't']

In [12]:
# Insert noisy observations (linked to no state) in the database
# Empty state will be denoted '_'
INSERTION_PROB = 0.05
(X_ni_train, y_ni_train), (X_ni_test, y_ni_test) = noisy_insertion(X_train, y_train, X_test, y_test, thresh_proba=INSERTION_PROB)

SAMPLE = 3
print("States sequence sample       : {}".format(y_ni_train[SAMPLE]))
print("Observations sequence sample : {}\n".format(X_ni_train[SAMPLE]))

states_ni, observations_ni = get_observations_states(X_ni_train, y_ni_train)
print("{} states :\n{}".format(len(states_ni), states_ni))
print("{} observations :\n{}".format(len(observations_ni), observations_ni))

States sequence sample       : ['a', 'c', 'c', 'o', '_', 'u', 'n', 't']
Observations sequence sample : ['a', 'c', 'v', 'o', 'o', 'u', 'n', 't']

27 states :
['_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
26 observations :
['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [13]:
# HMM first order
hmm = HMM(states_ni, observations_ni)
hmm.fit(X_ni_train, y_ni_train)
y_ni_test_pred = hmm.predict(X_ni_test)
display_correction_stats(X_ni_test, y_ni_test, y_ni_test_pred, name="\nHMM1")

1st order HMM created with: 
 * 27 states
 * 26 observations
Training initial states probabilities... Done.
Training transitions probabilities given states... Done.
Training observations probabilities given states... Done.

HMM1 score on test set
 * accuracy on full words : 61.69%
 * accuracy on letters    : 88.04%
   > typos corrected      : 328 (4.27%)
   > typos not corrected  : 784 (10.20%)
   > typos added          : 135 (1.76%)

Dummy score on test set
 * accuracy on full words : 51.63%
 * accuracy on letters    : 85.53%
   > typos corrected      : 0 (0.00%)
   > typos not corrected  : 1112 (14.47%)
   > typos added          : 0 (0.00%)


In [14]:
# HMM second order
hmm2 = HMM2(states_ni, observations_ni)
hmm2.fit(X_ni_train, y_ni_train)
y_ni_test_pred = hmm2.predict(X_ni_test)
display_correction_stats(X_ni_test, y_ni_test, y_ni_test_pred, name="\nHMM2")

2nd order HMM created with: 
 * 27 states
 * 26 observations
Training initial states probabilities... Done.
Training transitions probabilities given states... Done.
Training observations probabilities given states... Done.

HMM2 score on test set
 * accuracy on full words : 69.55%
 * accuracy on letters    : 89.70%
   > typos corrected      : 538 (7.00%)
   > typos not corrected  : 574 (7.47%)
   > typos added          : 218 (2.84%)

Dummy score on test set
 * accuracy on full words : 51.63%
 * accuracy on letters    : 85.53%
   > typos corrected      : 0 (0.00%)
   > typos not corrected  : 1112 (14.47%)
   > typos added          : 0 (0.00%)


In [15]:
# HMM second order with smoothing
hmm2 = HMM2(states_ni, observations_ni)
hmm2.fit(X_ni_train, y_ni_train, smoothing=True)
y_ni_test_pred = hmm2.predict(X_ni_test)
display_correction_stats(X_ni_test, y_ni_test, y_ni_test_pred, name="\nHMM2_smooth")

2nd order HMM created with: 
 * 27 states
 * 26 observations
Training initial states probabilities... Done.
Training transitions probabilities given states... Done.
Training observations probabilities given states... Done.

HMM2_smooth score on test set
 * accuracy on full words : 64.09%
 * accuracy on letters    : 89.40%
   > typos corrected      : 390 (5.07%)
   > typos not corrected  : 722 (9.39%)
   > typos added          : 93 (1.21%)

Dummy score on test set
 * accuracy on full words : 51.63%
 * accuracy on letters    : 85.53%
   > typos corrected      : 0 (0.00%)
   > typos not corrected  : 1112 (14.47%)
   > typos added          : 0 (0.00%)


**Conclusion :**

Résultats assez décevants... En réalité, si on observe le comportement du modèle en présence de seules fautes d'insertions (et non insertions + substitutions), on remarque que les performance de celui-ci sont à peine meilleures que le *Dummy*. Ceci est probablement dû aux peu de données d'apprentissage disponibles pour ce type d'erreurs.

# V - Omission de caractères

Pour l'omission (délétion) de caractères, on est dans le cas contraire, où une observation est manquante. On ne peut simplement rajouter une observation '\_', puisque cela reviendrait à indiquer directement au modèle qu'il y a eu une omission.  On serait alors dans un cas plus simple, mais où le modèle n'aurait pas à déterminer tout seul la résence d'une omission de caractère. Pour ceci, on modifie plutôt le corpus en supprimant certaines observations, et en autorisant le remplacement d'un observation par un état "double", composé de deux lettres. On se retrouve face à un problème beaucoup plus complexe, de 702 états (26 états simples + 676 états doubles) et 26 observations.

Exemple:
- **séquence d'états        :** ['a', 'c', 'c', 'o', 'un', 't']
- **séquence d'observations :** ['a', 'c', 'v', 'o', 'u', 't']

Remarque: de nombreuses lignes de la matrice de transition seront vides, puisque chacune de ces fautes est rarement observée.

In [16]:
def cross_states(states):
    """ Take a state list as input and return a cross state list
    (each new state being a concatenation of two states from states, or a unique state)
    Ex: If states=['a', 'b'], cross_states is ['a', 'b', 'aa', 'ab', 'ba', 'bb']

    :param states, list of string
    :return cross_states: states, list of string
    """
    
    cross_states = []
    
    for s1 in states:
        for s2 in states:
            cross_states.append(s1 + s2)
            
    return states + cross_states

In [17]:
# Remove some observations in the database
# To do that, we add the state without observation to the previous state
DELETION_PROB = 0.05
(X_no_train, y_no_train), (X_no_test, y_no_test) = noisy_omission(X_train, y_train, X_test, y_test, thresh_proba=DELETION_PROB)

states_no = cross_states(states)
observations_no = observations

print("{} states :\n{}".format(len(states_no), states_no))
print("{} observations :\n{}".format(len(observations_no), observations_no))

702 states :
['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'aa', 'ab', 'ac', 'ad', 'ae', 'af', 'ag', 'ah', 'ai', 'aj', 'ak', 'al', 'am', 'an', 'ao', 'ap', 'aq', 'ar', 'as', 'at', 'au', 'av', 'aw', 'ax', 'ay', 'az', 'ba', 'bb', 'bc', 'bd', 'be', 'bf', 'bg', 'bh', 'bi', 'bj', 'bk', 'bl', 'bm', 'bn', 'bo', 'bp', 'bq', 'br', 'bs', 'bt', 'bu', 'bv', 'bw', 'bx', 'by', 'bz', 'ca', 'cb', 'cc', 'cd', 'ce', 'cf', 'cg', 'ch', 'ci', 'cj', 'ck', 'cl', 'cm', 'cn', 'co', 'cp', 'cq', 'cr', 'cs', 'ct', 'cu', 'cv', 'cw', 'cx', 'cy', 'cz', 'da', 'db', 'dc', 'dd', 'de', 'df', 'dg', 'dh', 'di', 'dj', 'dk', 'dl', 'dm', 'dn', 'do', 'dp', 'dq', 'dr', 'ds', 'dt', 'du', 'dv', 'dw', 'dx', 'dy', 'dz', 'ea', 'eb', 'ec', 'ed', 'ee', 'ef', 'eg', 'eh', 'ei', 'ej', 'ek', 'el', 'em', 'en', 'eo', 'ep', 'eq', 'er', 'es', 'et', 'eu', 'ev', 'ew', 'ex', 'ey', 'ez', 'fa', 'fb', 'fc', 'fd', 'fe', 'ff', 'fg', 'fh', 'fi', 'fj', 'fk', 'fl', 'fm'

In [21]:
sample = 3
print(X_no_train[sample])
print(y_no_train[sample])

['c', 'v', 'o', 'u', 'n', 't']
['ac', 'c', 'o', 'u', 'n', 't']


In [19]:
# HMM first order
hmm = HMM(states_no, observations_no)
hmm.fit(X_no_train, y_no_train)
y_no_test_pred = hmm.predict(X_no_test)
display_correction_stats(X_no_test, y_no_test, y_no_test_pred)

1st order HMM created with: 
 * 702 states
 * 26 observations
Training initial states probabilities... Done.
Training transitions probabilities given states... Done.
Training observations probabilities given states... Done.
HMM score on test set
 * accuracy on full words : 62.89%
 * accuracy on letters    : 88.96%
   > typos corrected      : 290 (4.17%)
   > typos not corrected  : 699 (10.06%)
   > typos added          : 68 (0.98%)

Dummy score on test set
 * accuracy on full words : 52.83%
 * accuracy on letters    : 85.77%
   > typos corrected      : 0 (0.00%)
   > typos not corrected  : 989 (14.23%)
   > typos added          : 0 (0.00%)


**/!\ Attention ! /!\**

Attention, dans le cas de l'ordre 2, les matrices considérées peuvent prendre des tailles très importantes. Ici, la matrice de transition est de taille 702 x 702 x 702, ce qui dans le cas de flottants sur 64 bits (type de base d'un array numpy), revient à une matrice de plus de 2.5 Go. Afin d'éviter des erreurs de mémoire et limiter les calculs, la précision des nombres flottants est automatiquement modifiée en fonction de la dimension du problème, aboutissant ici à des matrices de "seulement" 600 Mo. Pour compenser cette perte de pécision, l'usage des log-probabilités est fortement recommandé (et effectué ici), permettant de mieux exploiter la plage des valeurs possibles, et ainsi limiter les erreurs de représentations.

Malgré l'utilisation du *broadcasting* numpy, l'algorithme de Viterbi reste très lent, avec plus d'une dizaine de secondes par séquence.

In [20]:
# HMM second order
hmm2 = HMM2(states_no, observations_no)
hmm2.fit(X_no_train, y_no_train)
y_no_test_pred = hmm2.predict(X_no_test)
display_correction_stats(X_no_test, y_no_test, y_no_test_pred)

2nd order HMM created with: 
 * 702 states
 * 26 observations
Training initial states probabilities... Done.
Training transitions probabilities given states... Done.
Training observations probabilities given states... Done.


In [ ]:
# HMM second order with smoothing
hmm2s = HMM2(states_no, observations_no)
hmm2s.fit(X_no_train, y_no_train, smoothing=True)
y_no_test_pred = hmm2s.predict(X_no_test)
display_correction_stats(X_no_test, y_no_test, y_no_test_pred)